In [1]:
import os
import sys
os.environ["JAVA_HOME"] = "/home/jovyan/ivanova/env39_sim4rec_demo_new/lib/jvm"
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["OMP_NUM_THREADS"] = "32"
os.environ["NUMBA_NUM_THREADS"] = "4"

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pyspark.sql import SparkSession

from sim4rec.modules import EncoderEstimator, evaluate_synthetic, SDVDataGenerator
from sim4rec.utils import pandas_to_spark

In [3]:
SPARK_LOCAL_DIR =  "./tmp"

spark = SparkSession.builder\
    .appName('simulator')\
    .master('local[4]')\
    .config('spark.sql.shuffle.partitions', '4')\
    .config('spark.default.parallelism', '4')\
    .config('spark.driver.extraJavaOptions', '-XX:+UseG1GC')\
    .config('spark.executor.extraJavaOptions', '-XX:+UseG1GC')\
    .config('spark.sql.autoBroadcastJoinThreshold', '-1')\
    .config('spark.driver.memory', '4g')\
    .config('spark.local.dir', SPARK_LOCAL_DIR)\
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

24/09/12 14:31:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/12 14:31:31 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
24/09/12 14:31:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/12 14:31:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
pdf = pd.DataFrame(
    data=np.random.normal(0, 1, size=(1000, 50)),
    columns=[f'attr_{i}' for i in range(50)]
)
pdf.insert(0, 'id', np.arange(1000))

df = pandas_to_spark(pdf)
df.toPandas()

,id,attr_0,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,...,attr_40,attr_41,attr_42,attr_43,attr_44,attr_45,attr_46,attr_47,attr_48,attr_49
0,0,0.340259,-1.153554,0.784389,1.793672,0.329759,1.196388,0.957169,-1.139086,-0.094718,...,0.437067,1.105563,0.966471,-0.041881,-0.991950,1.200777,0.245898,0.795817,0.875401,1.194018
1,1,-0.411738,1.924609,-1.586690,-0.892209,0.825743,-1.606956,-0.065935,0.675548,-0.908272,...,0.456097,1.291299,1.878074,-0.314440,-1.010161,-0.362480,1.528341,-0.979149,0.727207,1.164411
2,2,-0.070106,0.212639,-0.849616,-0.616477,2.760519,-0.728286,-0.894225,0.513202,2.140691,...,2.079152,-0.560622,-0.743466,0.918912,1.674549,0.060556,0.146445,-0.716262,-0.190587,0.745384
3,3,0.804773,1.091310,0.998837,-1.030688,1.579426,-0.814025,0.721834,0.342874,-1.225883,...,-0.128698,2.058802,-1.642637,-0.451542,-0.024078,-1.451718,-1.460769,-0.657520,0.136040,-1.198824
4,4,0.876635,-0.898820,-0.104922,1.719536,0.975224,-0.101548,-0.444754,-0.710269,-2.188647,...,-0.346856,-2.233627,-0.631779,-1.721387,-2.115973,-0.883213,-1.088403,0.297145,-1.238631,-0.945687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,-0.720816,-1.389677,1.546411,-0.570316,-0.378498,0.033477,0.591392,-0.805687,0.281293,...,1.578624,0.252061,1.137307,-0.849237,0.412419,0.532595,-0.619319,0.064593,1.494097,-4.144980
996,996,-0.635314,-2.034623,-1.127905,-0.422252,0.137656,0.536317,0.625832,0.724764,0.909819,...,1.782017,-0.561734,0.580766,0.055682,-1.007464,0.318983,0.077105,-0.972925,-0.430922,1.245628
997,997,0.625303,-1.278778,-0.929351,-0.650523,-1.420000,-0.674750,-0.042467,-0.199832,-1.759314,...,-1.118333,0.890766,0.080794,-0.130078,-0.078002,-1.751738,0.856002,1.554646,0.984155,-0.443692
998,998,1.296473,0.258232,-1.157472,-0.177413,-0.741170,-0.870851,-0.544458,0.135087,0.706782,...,-0.361690,1.644579,0.365380,-0.867129,1.873921,-1.594078,-0.486860,0.713616,-0.433612,1.255731


In [5]:
LATENT_DIM = 10

encoder = EncoderEstimator(
    inputCols=[f'attr_{i}' for i in range(50)],
    outputCols=[f'emb_{i}' for i in range(LATENT_DIM)],
    hidden_dim=16,
    lr=0.001,
    batch_size=64,
    num_loader_workers=4,
    max_iter=50,
    device_name='cpu'
)

encoder_model = encoder.fit(df)
encoder_model.transform(df.limit(10)).toPandas()

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9
0,0.683392,-0.392235,0.245129,-0.200656,-1.512890,-0.919066,-1.278216,-1.661627,0.643198,0.275999
1,0.360012,-0.010955,0.635952,-0.331309,-1.756720,-0.769607,-0.040503,-1.621125,-0.221358,-0.469650
2,1.886414,-1.169931,-0.629542,-1.532483,-1.603817,-0.827005,-0.075151,-2.748541,-0.134374,-0.967380
3,0.865490,-0.610279,-1.046842,-0.456238,-0.902792,-1.648435,-0.285958,-2.529303,0.053507,-0.367898
4,1.529015,0.037883,-0.008423,-0.244252,-1.294029,-1.035690,-0.328548,-2.957340,-0.125689,-0.160084
5,1.968582,-0.587480,-0.166715,-0.685445,-0.680951,-1.172747,-0.800314,-2.108650,-0.257105,-0.786563
6,0.609893,-0.696994,0.575391,-0.390636,-0.910702,-0.578762,-0.484948,-1.497815,0.648656,-0.131265
7,1.379513,0.370473,-0.227086,-1.142984,-2.141406,-0.448194,-2.059789,-1.971001,0.216287,0.191827
8,0.705079,-0.024583,-0.543103,0.111002,-2.546822,-0.766019,-1.204267,-1.638512,0.174755,-0.498766
9,1.931378,-0.543744,-1.380973,-0.009801,-1.690374,-0.789514,-1.208251,-2.897413,-0.473321,-0.089085


In [6]:
generator = SDVDataGenerator(
    label='synth',
    id_column_name='id',
    parallelization_level=4,
    seed=42
)
generator.fit(encoder_model.transform(df))

generator.generate(10).toPandas()

,id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9
0,synth_0,1.421952,-0.664799,-0.113525,-0.579171,-1.744761,-0.663123,-0.439580,-4.332823,-1.271706,-0.618492
1,synth_1,1.472361,0.157253,-0.248346,-0.946558,-2.546917,-0.664514,-0.530075,-2.271936,-0.830395,-0.007163
2,synth_2,1.308155,0.235449,-1.419100,-1.097231,-1.618757,-0.874752,-0.665742,-1.831077,0.534130,-0.563267
3,synth_3,1.265597,-0.089828,0.501277,-1.230010,-1.259124,-1.166757,-1.889318,-3.195850,-0.792598,0.183104
4,synth_4,0.859793,0.376751,-0.528271,-0.427108,-0.512386,-1.190079,-1.560926,-2.299077,-0.789706,-0.750679
5,synth_5,1.326637,-0.095965,-0.307343,-0.535825,-1.353476,-0.565261,-0.774362,-1.886851,-0.453719,0.524455
6,synth_6,0.913485,0.320262,0.162011,0.163208,-1.370468,-0.180885,-0.846993,-2.353354,-1.026272,-0.517754
7,synth_7,0.723497,-0.077613,-0.042598,-0.317570,-0.885523,0.001560,-0.913156,-2.285257,-0.154805,-0.626550
8,synth_8,1.300860,0.089597,-0.179966,-0.798016,-0.684374,-1.322987,-0.593257,-1.920334,0.431156,0.281663
9,synth_9,1.187022,-0.137912,0.011729,-0.298395,-1.013820,-1.723797,-0.704754,-1.314424,-0.332011,-0.826291


In [7]:
evaluate_synthetic(generator.generate(1000).drop('id'), encoder_model.transform(df))

{'LogisticDetection': 1.0,
 'SVCDetection': 0.9982775568361154,
 'KSTest': 0.9652000000000001,
 'ContinuousKLDivergence': 0.8304997964186596}